In [9]:
import nltk
from collections import defaultdict
from array import array
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import math
import numpy as np
import collections
from numpy import linalg as la
import json
import re
import string
#import emoji

In [10]:
path = 'dataset_tweets_WHO.txt'

#convert the text to json
with open(path) as f:
    tweets_json = json.load(f)
    

In [11]:
def remove_punct(line):
    """
    Helper function to remove punctuation
    
    Arugment:
    line -- string of text
    
    Returns:
    line -- string of text without punctuation
    """
    return line.translate(str.maketrans('', '', string.punctuation.replace('#', '')))

def build_terms(line):
    """
    Preprocess the article text (title + body) by removing stop words, emojis, and punctuation and
    stemming, transforming to lowercase and returning the tokens of the text.
    
    Argument:
    line -- string (text) to be preprocessed
    
    Returns:
    line -- a list of tokens corresponding to the input text after the preprocessing
    """

    stemmer = PorterStemmer()
    stop_words = set(stopwords.words("english"))
    ## START CODE
    line =  line.lower() ## Transform in lowercase
    line = "".join(c for c in line if c in string.printable) ## remove non-ASCII terms like emojis and symbols
    line = remove_punct(line) ## remove punctuation
    line = line.split() ## Tokenize the text to get a list of terms
    
    line = [word for word in line if not (re.match("^qampa$" , word) or re.match("^amp$" , word) or re.match("^http" , word)) 
    and word] #remove html tags, blank spaces like '', and urls
    line = [word for word in line if not word.isnumeric()]
    
    line = [word for word in line if word not in stop_words] ##eliminate the stopwords
    line = [stemmer.stem(word) for word in line] ## perform stemming
    
    # add unhashtagged word if it's hashtag is present 
    # e.x. if #covid is present, we also add covid as a token
    line = line + [word.replace('#', '') for word in line if word[0] == '#' ] 
    
    line = list(set(line)) #remove duplicates
    """to discuss, should we remove hashtags? 
    yes because if we don't, then something e.x. #covid and covid will have different term frequencies. A document that only
    contains '#covid' is probably very relevant for covid, but will get a tf of 0 if we are searching using the query 'covid'
    """
    
    #should we remove numbers? also ellipses are still present
    #also need to lemmatize words
    return line

In [12]:
for key in tweets_json:
    key = str(key)
    tweets_json[key]['full_text'] = build_terms(tweets_json[key]['full_text'])

In [14]:
#inspect the first couple of tweets to check
for i in range(10):
    print(tweets_json[str(i)]['full_text'])

['openwho', 'intern', 'respons', 'equip', 'work', 'today', 'learn', 'ready4respons', 'launch', 'public', 'risk', 'curriculum', 'core', 'within', 'disast', '#openwho', 'multiti', '#ready4respons', 'need', 'day', 'reduct', 'compet', 'health', 'emerg', 'start', 'help']
['commun', 'weak', 'popul', 'live', 'like', 'especi', 'shown', 'entir', 'vulner', 'migrant', 'indigen', 'condit', 'system', 'disast', 'peopl', 'fragil', 'covid19', '#covid19', 'humanitarian', 'health', 'emerg', 'affect']
['strong', 'group', 'intern', 'respond', 'invest', 'equit', 'readi', 'system', 'better', 'risk', 'countri', 'care', 'resili', 'disast', 'must', 'marginalis', 'access', 'day', 'protect', 'achiev', 'reduct', 'health', 'emerg', 'suppli', 'gender', 'ensur', 'equiti']
['congratul', 'reach', '#algeria', '16th', 'africa', 'whoafro', 'rt', '#africa', 'pop', 'fulli', 'mileston', 'countri', 'vaccin', 'algeria']
['esperando', 'completament', 'todava', 'covid19', 'vacunado', 'pued', 'contraer', 'rt', 'opsom', 'importa'